## How do we Fix Density Fragmentation

We could fix this with INDEX REORG 

REORG ALTER INDEX \[ClusteredIndex\_1fba0db5c48b40288124497ec2198389\] ON \[dbo\].\[FactFinance100m\] REORGANIZE

But, lets introduce the GitHub Library function ColumnStoreOptimize which can locate all Column Stores with low density and REORG them [https://github.com/ProdataSQL/SynapseTools/tree/main/SqlPools/Maintenance](https://github.com/ProdataSQL/SynapseTools/tree/main/SqlPools/Maintenance)

In [3]:
/*  ALTER INDEX ClusteredIndex_1fba0db5c48b40288124497ec2198389 ON [dbo].[FactFinance100m] REORGANIZE */
exec   [dbo].[ColumnstoreOptimize]  @Tables='FactFinance100m'
,@DensityThreshold=25 /* Default=25 */
,@OpenThreshold=null
,@DeleteThreshold=null
,@TimeLimit =null
, @Execute='Y'


Date and time: 2022-07-08 12:59:40

Server: aw-dev

Version: 10.0.15665.0

Edition: SQL Azure

Parameters: @Tables= 'FactFinance100m', @DensityThreshold = 25.00, @OpenThreshold = 200000, @DeleteThreshold = 200000, @Execute = Y

Version: Microsoft Azure SQL Data Warehouse - 10.0.15665.0 Jul 7 2022 07:03:56 Copyright (c) Microsoft Corporation

Source: https://github.com/ProdataSQL/SynapseTools/tree/main/SqlPools/Maintenance

Date and time: 2022-07-08 12:59:42

SqlCommand: ALTER INDEX ClusteredIndex_766ace22025346dba8b1db2efab6ee27 ON [dbo].[FactFinance100m] REORGANIZE

Total execution time: 00:00:13.103

In [4]:
/*  Now we can check commandlog and also imprves Stats
    From 180 row groups to just 60 (one per distibution)
 */
SELECT TOP 1 * FROM dbo.CommandLog ORDER BY StartTime DESC
SELECT * From vColumnstoreStats where table_name ='factFinance100m'


(1 row affected)

(1 row affected)

Total execution time: 00:00:01.439

ID,DatabaseName,SchemaName,ObjectName,ObjectType,IndexName,IndexType,StatisticsName,PartitionNumber,ExtendedInfo,Command,CommandType,StartTime,EndTime,ErrorNumber,ErrorMessage
86,AdventureWorksDW,dbo,FactFinance100m,U,ClusteredIndex_e2f1410ade274620bfb52e91dcee6ddb,5,NULL,NULL,<ExtendedInfo><RowGroups>420</RowGroups><Rows>230000000</Rows><OpenRows>4170880</OpenRows><DeletedRows>0</DeletedRows><DensityFragmentation>48.72 %</DensityFragmentation></ExtendedInfo>,ALTER INDEX ClusteredIndex_e2f1410ade274620bfb52e91dcee6ddb ON [dbo].[FactFinance100m] REORGANIZE WITH (COMPRESS_ALL_ROW_GROUPS = ON),ALTER INDEX,2022-07-08 14:33:59.787,2022-07-08 14:34:07.160,NULL,NULL


execution_date,database_name,schema_name,table_name,partition_number,partition_scheme,object_id,index_name,row_count,deleted_row_count,row_group_count,compressed_row_count,compressed_rowgroup_count,open_rowgroup_count,open_row_count,compressed_row_max,compressed_row_avg,fragmentation_density,fragmentation_deletes,fragmentation_open
2022-07-08 14:35:45.453,AdventureWorksDW,dbo,FactFinance100m,NULL,NULL,290868153,ClusteredIndex_e2f1410ade274620bfb52e91dcee6ddb,230000000,0,300,230000000,300,0,0,1048576,766666,26.8900,0.0000,0.0000
